# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
#load python packages
import os #op sys access
import pandas as pd
import datetime  #date work
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn import preprocessing #preprocessing package
from sklearn.model_selection import train_test_split #train_test_split package
# for modeling
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

#check path
os.getcwd()

'C:\\Users\\Hailey\\JNote'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
#change working directory and check what is in it
path = 'C:\\Users\\Hailey\\Documents\\temp' #working path
dataPathIn = 'C:\\Users\\Hailey\\Documents\\temp\\data\\step3_output.csv' #data path
os.chdir(path) 
path = os.getcwd() 
os.listdir()

#load the data
df = pd.read_csv(dataPathIn)
#df.info()

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

A code segment for this is shown in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

#### Features for model 1

In [3]:
dfo=df[['state']] # select object column 'state'
df1 = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1) #add dummies to df with names like 'state_...'

In [4]:
print(dfo.head())
print(df1.info())

        state
0      Alaska
1     Arizona
2  California
3  California
4  California
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 60 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  171 non-null    object 
 1   summit_elev           171 non-null    int64  
 2   vertical_drop         171 non-null    int64  
 3   trams                 171 non-null    int64  
 4   fastEight             171 non-null    int64  
 5   fastSixes             171 non-null    int64  
 6   fastQuads             171 non-null    int64  
 7   quad                  171 non-null    int64  
 8   triple                171 non-null    int64  
 9   double                171 non-null    int64  
 10  surface               171 non-null    int64  
 11  total_chairs          171 non-null    int64  
 12  Runs                  171 non-null    int64  
 13  TerrainParks          171 non-null    int

#### Features for model 2

In [5]:
df2 = df.drop('state', axis = 1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               171 non-null    object 
 1   summit_elev        171 non-null    int64  
 2   vertical_drop      171 non-null    int64  
 3   trams              171 non-null    int64  
 4   fastEight          171 non-null    int64  
 5   fastSixes          171 non-null    int64  
 6   fastQuads          171 non-null    int64  
 7   quad               171 non-null    int64  
 8   triple             171 non-null    int64  
 9   double             171 non-null    int64  
 10  surface            171 non-null    int64  
 11  total_chairs       171 non-null    int64  
 12  Runs               171 non-null    int64  
 13  TerrainParks       171 non-null    int64  
 14  LongestRun_mi      171 non-null    float64
 15  SkiableTerrain_ac  171 non-null    float64
 16  Snow Making_ac     171 non

#### Features for model 3

In [6]:
df3 = df.drop(['state', 'summit_elev', 'trams', 'fastEight', 'fastSixes', 'fastQuads','quad','triple','double','surface','total_chairs', 'Snow Making_ac','yearsOpen','averageSnowfall'],axis = 1)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               171 non-null    object 
 1   vertical_drop      171 non-null    int64  
 2   Runs               171 non-null    int64  
 3   TerrainParks       171 non-null    int64  
 4   LongestRun_mi      171 non-null    float64
 5   SkiableTerrain_ac  171 non-null    float64
 6   daysOpenLastYear   171 non-null    int64  
 7   AdultWeekday       171 non-null    float64
 8   AdultWeekend       171 non-null    float64
 9   projectedDaysOpen  171 non-null    int64  
 10  NightSkiing_ac     171 non-null    float64
 11  clusters           171 non-null    int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 16.2+ KB


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X1 = df1.drop(['Name','AdultWeekend'], axis=1)
X2 = df2.drop(['Name','AdultWeekend'], axis=1)
X3 = df3.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.loc[:,'AdultWeekend']
y2 = df.loc[:,'AdultWeekday']
y3 = df1.loc[:, 'daysOpenLastYear']
y4 = df1.loc[:,'projectedDaysOpen']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler1 = preprocessing.StandardScaler().fit(X1)
scaler2 = preprocessing.StandardScaler().fit(X2)
scaler3 = preprocessing.StandardScaler().fit(X3)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X1_scaled=scaler1.transform(X1)
X2_scaled=scaler2.transform(X2)
X3_scaled=scaler3.transform(X3)
#X1_scaled

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.   Then, we will use other parameters in y.</font>**

In [8]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()
y2 = y2.ravel()
y3 = y3.ravel()
y4 = y4.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X1_train, X1_test, y_train, y_test = train_test_split(X1_scaled, y, test_size=0.25, random_state=1)
X1_train2, X1_test2, y_train2, y_test2 = train_test_split(X1_scaled, y2, test_size=0.25, random_state=1)
X1_train3, X1_test3, y_train3, y_test3 = train_test_split(X1_scaled, y3, test_size=0.25, random_state=1)
X1_train4, X1_test4, y_train4, y_test4 = train_test_split(X1_scaled, y4, test_size=0.25, random_state=1)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y, test_size=0.25, random_state=1)
X2_train2, X2_test2, y2_train2, y2_test2 = train_test_split(X2_scaled, y2, test_size=0.25, random_state=1)
X2_train3, X2_test3, y2_train3, y2_test3 = train_test_split(X2_scaled, y3, test_size=0.25, random_state=1)
X2_train4, X2_test4, y2_train4, y2_test4 = train_test_split(X2_scaled, y4, test_size=0.25, random_state=1)


X3_train, X3_test, y3_train, y3_test = train_test_split(X3_scaled, y, test_size=0.25, random_state=1)
X3_train2, X3_test2, y3_train2, y3_test2 = train_test_split(X3_scaled, y2, test_size=0.25, random_state=1)
X3_train3, X3_test3, y3_train3, y3_test3 = train_test_split(X3_scaled, y3, test_size=0.25, random_state=1)
X3_train4, X3_test4, y3_train4, y3_test4 = train_test_split(X3_scaled, y4, test_size=0.25, random_state=1)


Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
#all first model set
lm = linear_model.LinearRegression()
model_x1y = lm.fit(X1_train,y_train)
model_x12y2 = lm.fit(X1_train2,y_train2)
model_x13y3 = lm.fit(X1_train3,y_train3)
model_x14y4 = lm.fit(X1_train4,y_train4)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [10]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred1 = model_x1y.predict(X1_test)
y_pred12 = model_x12y2.predict(X1_test2)
y_pred13 = model_x13y3.predict(X1_test3)
y_pred14 = model_x14y4.predict(X1_test4)

## Review Model Outcomes — Iterate over additional models as needed

In [11]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
print(explained_variance_score(y_test, y_pred1))
print(explained_variance_score(y_test2, y_pred12))
print(explained_variance_score(y_test3, y_pred13))
print(explained_variance_score(y_test4, y_pred14))

-0.3396914438501495
-0.1608920136352694
0.406098121676697
1.0


In [12]:
print(mean_absolute_error(y_test, y_pred1))
print(mean_absolute_error(y_test2, y_pred12))
print(mean_absolute_error(y_test3, y_pred13))
print(mean_absolute_error(y_test4, y_pred14))

52.92635325498042
60.14532861476239
12.11627906976745
2.3464434529752146e-14


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [13]:
lm.intercept_

110.86549707602339

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [14]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
pd.DataFrame(abs(lm.coef_), X1.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(25)

,Coefficient
projectedDaysOpen,21.670567
trams,0.315977
fastEight,0.251391
fastSixes,0.200836
total_chairs,0.047651
double,0.025272
surface,0.024910
triple,0.021247
quad,0.012111
fastQuads,0.010139


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [15]:
#all second model set
model_x2y2 = lm.fit(X2_train,y2_train)
model_x22y22 = lm.fit(X2_train2,y2_train2)
model_x23y23 = lm.fit(X2_train3,y2_train3)
model_x24y24 = lm.fit(X2_train4,y2_train4)

In [16]:
y_pred2 = model_x2y2.predict(X2_test)
y_pred22 = model_x22y22.predict(X2_test2)
y_pred23 = model_x23y23.predict(X2_test3)
y_pred24 = model_x24y24.predict(X2_test4)

In [17]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
print(explained_variance_score(y2_test, y_pred2))
print(explained_variance_score(y2_test2, y_pred22))
print(explained_variance_score(y2_test3, y_pred23))
print(explained_variance_score(y2_test4, y_pred24))

-0.3396914438501486
-0.16089201363526828
0.4060981216766987
1.0


In [18]:
print(mean_absolute_error(y2_test, y_pred2))
print(mean_absolute_error(y2_test2, y_pred22))
print(mean_absolute_error(y2_test3, y_pred23))
print(mean_absolute_error(y2_test4, y_pred24))

52.92635325498042
60.14532861476239
12.116279069767439
2.115103957611461e-14


In [19]:
pd.DataFrame(abs(lm.coef_), X2.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(25)

,Coefficient
projectedDaysOpen,2.167057e+01
daysOpenLastYear,2.240083e-14
LongestRun_mi,1.679669e-14
triple,1.158223e-14
fastQuads,1.070342e-14
NightSkiing_ac,7.716326e-15
SkiableTerrain_ac,7.697114e-15
quad,6.208893e-15
yearsOpen,4.083904e-15
trams,3.552714e-15


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [20]:
#all third model set
model_x3y3 = lm.fit(X3_train,y3_train)
model_x32y32 = lm.fit(X3_train2,y3_train2)
model_x33y33 = lm.fit(X3_train3,y3_train3)
model_x34y34 = lm.fit(X3_train4,y3_train4)

In [21]:
y_pred3 = model_x3y3.predict(X3_test)
y_pred32 = model_x32y32.predict(X3_test2)
y_pred33 = model_x33y33.predict(X3_test3)
y_pred34 = model_x34y34.predict(X3_test4)

In [22]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
print(explained_variance_score(y3_test, y_pred3))
print(explained_variance_score(y3_test2, y_pred32))
print(explained_variance_score(y3_test3, y_pred33))
print(explained_variance_score(y3_test4, y_pred34))

-0.33969144385014927
-0.16089201363526984
0.406098121676697
1.0


In [23]:
print(mean_absolute_error(y3_test, y_pred3))
print(mean_absolute_error(y3_test2, y_pred32))
print(mean_absolute_error(y3_test3, y_pred33))
print(mean_absolute_error(y3_test4, y_pred34))

52.92635325498042
60.14532861476238
12.116279069767447
1.1236489774810886e-14


In [24]:
pd.DataFrame(abs(lm.coef_), X3.columns, columns=['Coefficient']).sort_values('Coefficient', ascending=False).head(25)

,Coefficient
projectedDaysOpen,2.167057e+01
AdultWeekday,9.735656e-15
clusters,9.567890e-15
NightSkiing_ac,5.160587e-15
TerrainParks,5.038966e-15
LongestRun_mi,4.968762e-15
SkiableTerrain_ac,3.227714e-15
daysOpenLastYear,1.775554e-15
vertical_drop,1.014651e-15
Runs,6.009747e-16


In [25]:
print(X1.columns)
print(X2.columns)
print(X3.columns)

Index(['summit_elev', 'vertical_drop', 'trams', 'fastEight', 'fastSixes',
       'fastQuads', 'quad', 'triple', 'double', 'surface', 'total_chairs',
       'Runs', 'TerrainParks', 'LongestRun_mi', 'SkiableTerrain_ac',
       'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen', 'averageSnowfall',
       'AdultWeekday', 'projectedDaysOpen', 'NightSkiing_ac', 'clusters',
       'state_Alaska', 'state_Arizona', 'state_California', 'state_Colorado',
       'state_Connecticut', 'state_Idaho', 'state_Illinois', 'state_Indiana',
       'state_Iowa', 'state_Maine', 'state_Maryland', 'state_Massachusetts',
       'state_Michigan', 'state_Minnesota', 'state_Missouri', 'state_Montana',
       'state_Nevada', 'state_New Hampshire', 'state_New Jersey',
       'state_New Mexico', 'state_New York', 'state_North Carolina',
       'state_Ohio', 'state_Oregon', 'state_Pennsylvania',
       'state_Rhode Island', 'state_South Dakota', 'state_Tennessee',
       'state_Utah', 'state_Vermont', 'state_Virginia',

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Model Column| Explained Variance| Mean Absolute Error|Features Dropped|
| --- |--- |--- | --- | --- |
| Model 1. | Adult Weekend |-0.34 | 52.93 |-|
|  | Adult Weekday |-0.16 | 60.15 |-|
|  | Days Open Last Year |0.41 | 12.12 |-|
|  | Projected Days Open |1.0 | 2.35e-14 |-|
| Model 2. | Adult Weekend | -0.34|52.93 |'state'|
|  | Adult Weekday | -0.16| 60.15|'state'|
|  | Days Open Last Year | 0.41| 12.12|'state'|
|  | Projected Days Open | 1.0| 2.12e-14|'state'|
| Model 3. | Adult Weekend | -0.34 |52.93 |'state', 'summit_elev', 'trams', 'fastEight', 'fastSixes', 'fastQuads','quad','triple','double','surface','total_chairs', 'Snow Making_ac','yearsOpen','averageSnowfall'|
|  | Adult Weekday | -0.16 |60.15 |'state', 'summit_elev', 'trams', 'fastEight', 'fastSixes', 'fastQuads','quad','triple','double','surface','total_chairs', 'Snow Making_ac','yearsOpen','averageSnowfall'|
|  | Days Open Last Year | 0.41 |12.12 |'state', 'summit_elev', 'trams', 'fastEight', 'fastSixes', 'fastQuads','quad','triple','double','surface','total_chairs', 'Snow Making_ac','yearsOpen','averageSnowfall'|
|  | Projected Days Open| 1.0 |1.12e-15 |'state', 'summit_elev', 'trams', 'fastEight', 'fastSixes', 'fastQuads','quad','triple','double','surface','total_chairs', 'Snow Making_ac','yearsOpen','averageSnowfall'|

Model Selection:

The models show that minimal or no change despite removel of features.  This could be explained by the small sample size that resulted from removal of outliers.  Based on this analysis, feature removal has no significant effect.  